# Installing Packages

In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml==1.1.6
!pip install python-scipy

In [4]:
!pip install basemap
#!pip install libgeos-3.5.0
#!pip install libgeos-dev
#!pip install https://github.com/matplotlib/basemap/archive/master.zip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 940.3/940.3 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 144.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 141.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 125.0 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.9.2
    Uninstalling matplotlib-3.9.2:
      Successfully uninstalled matplotlib-3.9.2


# Importing packages

In [5]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
#import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from mpl_toolkits.basemap import Basemap
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")

# Reading all datasets from SF

In [6]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [8]:
# Define the table name
table_name = 'AIRLINES'

# Execute SQL query to retrieve the data from the specified table
df = my_session.sql("select * from {}".format(table_name))

# Convert the Snowflake DataFrame to a pandas DataFrame for easier manipulation and analysis
df = df.to_pandas()

# Display the data
df

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [ ]:
# Define the table name
table_name = 'AIRLINES'

# Execute SQL query to retrieve the data from the specified table
df = my_session.sql("select * from {}".format(table_name))

# Convert the Snowflake DataFrame to a pandas DataFrame for easier manipulation and analysis
df = df.to_pandas()

# Display the data
df

In [ ]:
# Define the table name
table_name = 'AIRLINES'

# Execute SQL query to retrieve the data from the specified table
df = my_session.sql("select * from {}".format(table_name))

# Convert the Snowflake DataFrame to a pandas DataFrame for easier manipulation and analysis
df = df.to_pandas()

# Display the data
df